In [ ]:
# getting the model from official first order model repo
!pip install PyYAML==5.3.1
!git clone https://github.com/AliaksandrSiarohin/first-order-model

In [ ]:
#getting a default image (my photo :)
!gdown --id 1bEzCUOM3-xL3IdK6KU5Ovk1UsF5QKHP6
# getting a default driving video
!gdown --id 1bX99o_MDz_vFVAQzcon3TS-eaMa3fPyw
#getting pretrained weights
!gdown --id 1_v_xW1V52gZCZnXgh1Ap_gwA9YVIzUnS


In [ ]:
#packages
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cd first-order-model

In [ ]:
from demo import make_animation, load_checkpoints
from skimage import img_as_ubyte

In [ ]:
## Load image and driving video
## You can upload your image and driving video and change these..

source_image = imageio.imread('../face.jpg')
driving_video = imageio.mimread('../bakamitai_template.mp4')

In [ ]:
## Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

In [ ]:
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
## Loading the pretrained model
generator, kp_detector = load_checkpoints(config_path='./config/vox-256.yaml', 
                            checkpoint_path='../vox-cpk.pth.tar')

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

## Save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])

## Video can be downloaded from /content folder
HTML(display(source_image, driving_video, predictions).to_html5_video())